In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [4]:
def readin_small_data():
    """Read small data set into a 2D numpy array
    return: numpy array contains the small data set, shape: (50,000, 230)
    """
    PATH = os.getcwd()
    DATA_PATH = os.path.join(PATH, 'small_clean.csv')
    
    data = pd.read_csv(DATA_PATH, delimiter=',')
    
    return data

In [5]:
data = readin_small_data()
print(data)

       Unnamed: 0          0
0               0  12.000000
1               1   2.000000
2               2  58.000000
3               3   0.000000
4               4   0.000000
5               5  10.000000
6               6  16.000000
7               7   2.000000
8               8   2.000000
9               9  24.000000
10             10   6.000000
11             11  28.000000
12             12   0.000000
13             13  22.000000
14             14   0.000000
15             15   0.000000
16             16  14.000000
17             17   0.000000
18             18   2.000000
19             19  14.000000
20             20   8.000000
21             21   0.000000
22             22  12.000000
23             23   0.000000
24             24  14.000000
25             25   0.000000
26             26   8.461026
27             27   4.000000
28             28   0.000000
29             29   2.000000
...           ...        ...
49970       49970   2.000000
49971       49971   8.461026
49972       49

In [6]:
del data['Unnamed: 0']

In [9]:
def readin_label(target):
    """Read upselling, churn or appetency label into a 2D numpy array
    parameter target: choose which label to load
    return: numpy array contains the specified label of small data set, shape: (50,000, 1)
    """
    assert target in ['upselling', 'churn', 'appetency']
    PATH = os.getcwd()
    LABEL_PATH = os.path.join(PATH, 'orange_small_train', 'orange_small_train_'+target+'.labels.txt')

    label = pd.read_csv(LABEL_PATH, delimiter='\t')
    label[label==1] = True
    label[label==-1] = False
    
    return label

upselling_label = readin_label('upselling')
print(upselling_label.shape)
churn_label = readin_label('churn')
print(churn_label.shape)
appetency_label = readin_label('appetency')
print(appetency_label.shape)

(49999, 1)
(49999, 1)
(49999, 1)


In [10]:
labels = np.hstack([appetency_label, churn_label, upselling_label])
labels = np.hstack([labels, ~np.any(labels, axis=1)[:,np.newaxis]])
labels = labels.astype(int)

In [16]:
(sklearn.preprocessing.robust_scale(data)).shape

(50000, 1)

In [11]:
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import keras
from sklearn.cross_validation import train_test_split
from keras import regularizers

model = Sequential()

model.add(Dense(units=100, activation='relu', input_dim=448, kernel_regularizer=regularizers.l2(0.01)))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
model.add(Dense(units=100, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
# model.add(Dense(units=100, activation='relu'))
# keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

model.add(Dense(units=4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])

# trainX, valX, trainY, valY = train_test_split(clean_numerical_data, labels, test_size=0.2, random_state=42)



model.fit(sklearn.preprocessing.robust_scale(data), labels, epochs=400, batch_size=200, validation_split=0.2)

Using TensorFlow backend.
/afs/inf.ed.ac.uk/user/s17/s1717961/miniconda2/envs/mlp/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/afs/inf.ed.ac.uk/user/s17/s1717961/miniconda2/envs/mlp/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


ValueError: Error when checking input: expected dense_1_input to have shape (448,) but got array with shape (1,)

In [14]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from multiprocessing import cpu_count
from sklearn.cross_validation import StratifiedShuffleSplit
from random import seed
from pandas import Categorical, concat, DataFrame, get_dummies, read_table, Series
RANDOM_SEED = 99
seed(RANDOM_SEED)
B = 1200

rf_model = RandomForestClassifier(
    n_estimators=B,
    criterion='entropy',
    max_depth=None,   # expand until all leaves are pure or contain < MIN_SAMPLES_SPLIT samples
    min_samples_split=100,
    min_samples_leaf=50,
    min_weight_fraction_leaf=0.0,
    max_features=None,   # number of features to consider when looking for the best split; None: max_features=n_features
    max_leaf_nodes=None,   # None: unlimited number of leaf nodes
    bootstrap=True,
    oob_score=True,   # estimate Out-of-Bag Cross Entropy
    n_jobs=cpu_count() - 2,   # paralellize over all CPU cores but 2
    class_weight=None,    # our classes are skewed, but but too skewed
    random_state=RANDOM_SEED,
    verbose=0,
    warm_start=False)

rf_model.fit(X=get_dummies(sklearn.preprocessing.robust_scale(data)), y=labels)

Exception: Data must be 1-dimensional

In [11]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [12]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.